In [8]:
start_date = '2016-01-01'
end_date = '2021-06-01'
target = "10d_ret_standscalar"
alpha_name = "AlphaNetV1_TsNorm30"
shift = 5
sequence = 6
LR = [0.001,0.0001,0.00001]
epoch_num = [30,30,30]

In [4]:
alpha_list = [
'open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover',
#             'open_ts_norm_30d',
#              'high_ts_norm_30d',
#              'low_ts_norm_30d',
#              'close_ts_norm_30d',
#              'vwap_ts_norm_30d',
#              'volume_ts_norm_30d',
#              'pct_chg_ts_norm_30d',
#              'turnover_ts_norm_30d',
#              'free_turnover_ts_norm_30d'
             ]
feat_list = alpha_list.copy()
method = ["COV","CORR","STD","ZSCORE","RETURN","DECAY"]
from AlphaNet.Data import generate_alpha_list
alpha_list = generate_alpha_list(feat_list,method,day=10)
alpha_list
# alpha_list = [i +"_residual" for i in feat_list]

['COV_open_high_10',
 'COV_open_low_10',
 'COV_open_close_10',
 'COV_open_vwap_10',
 'COV_open_volume_10',
 'COV_open_pct_chg_10',
 'COV_open_turnover_10',
 'COV_open_free_turnover_10',
 'COV_high_low_10',
 'COV_high_close_10',
 'COV_high_vwap_10',
 'COV_high_volume_10',
 'COV_high_pct_chg_10',
 'COV_high_turnover_10',
 'COV_high_free_turnover_10',
 'COV_low_close_10',
 'COV_low_vwap_10',
 'COV_low_volume_10',
 'COV_low_pct_chg_10',
 'COV_low_turnover_10',
 'COV_low_free_turnover_10',
 'COV_close_vwap_10',
 'COV_close_volume_10',
 'COV_close_pct_chg_10',
 'COV_close_turnover_10',
 'COV_close_free_turnover_10',
 'COV_vwap_volume_10',
 'COV_vwap_pct_chg_10',
 'COV_vwap_turnover_10',
 'COV_vwap_free_turnover_10',
 'COV_volume_pct_chg_10',
 'COV_volume_turnover_10',
 'COV_volume_free_turnover_10',
 'COV_pct_chg_turnover_10',
 'COV_pct_chg_free_turnover_10',
 'COV_turnover_free_turnover_10',
 'CORR_open_high_10',
 'CORR_open_low_10',
 'CORR_open_close_10',
 'CORR_open_vwap_10',
 'CORR_open_

In [5]:
alpha_list2 = alpha_list.copy()
alpha_list = []
for i in alpha_list2:
    alpha_list.append(i+"_TsNorm30d")
alpha_list

['COV_open_high_10_TsNorm30d',
 'COV_open_low_10_TsNorm30d',
 'COV_open_close_10_TsNorm30d',
 'COV_open_vwap_10_TsNorm30d',
 'COV_open_volume_10_TsNorm30d',
 'COV_open_pct_chg_10_TsNorm30d',
 'COV_open_turnover_10_TsNorm30d',
 'COV_open_free_turnover_10_TsNorm30d',
 'COV_high_low_10_TsNorm30d',
 'COV_high_close_10_TsNorm30d',
 'COV_high_vwap_10_TsNorm30d',
 'COV_high_volume_10_TsNorm30d',
 'COV_high_pct_chg_10_TsNorm30d',
 'COV_high_turnover_10_TsNorm30d',
 'COV_high_free_turnover_10_TsNorm30d',
 'COV_low_close_10_TsNorm30d',
 'COV_low_vwap_10_TsNorm30d',
 'COV_low_volume_10_TsNorm30d',
 'COV_low_pct_chg_10_TsNorm30d',
 'COV_low_turnover_10_TsNorm30d',
 'COV_low_free_turnover_10_TsNorm30d',
 'COV_close_vwap_10_TsNorm30d',
 'COV_close_volume_10_TsNorm30d',
 'COV_close_pct_chg_10_TsNorm30d',
 'COV_close_turnover_10_TsNorm30d',
 'COV_close_free_turnover_10_TsNorm30d',
 'COV_vwap_volume_10_TsNorm30d',
 'COV_vwap_pct_chg_10_TsNorm30d',
 'COV_vwap_turnover_10_TsNorm30d',
 'COV_vwap_free_turn

In [3]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
from AlphaNet.Data import concat_original_data , generate_shift_data

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


In [9]:
concat_original_data(alpha_name=alpha_name,alpha_list=alpha_list)

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template
['COV_open_high_10_TsNorm30d', 'COV_open_low_10_TsNorm30d', 'COV_open_close_10_TsNorm30d', 'COV_open_vwap_10_TsNorm30d', 'COV_open_volume_10_TsNorm30d', 'COV_open_pct_chg_10_TsNorm30d', 'COV_open_turnover_10_TsNorm30d', 'COV_open_free_turnover_10_TsNorm30d', 'COV_high_low_10_TsNorm30d', 'COV_high_close_10_TsNorm30d', 'COV_high_vwap_10_TsNorm30d', 'COV_high_volume_10_TsNorm30d', 'COV_high_pct_chg_10_TsNorm30d', 'COV_high_turnover_10_TsNorm30d', 'COV_high_free_turnover_10_TsNorm30d', 'COV_low_close_10_TsNorm30d', 'COV_low_vwap_10_TsNorm30d', 'COV_low_volume_10_TsNorm30d', 'COV_low_pct_chg_10_TsNorm30d', 'COV_low_turnover_10_TsNorm30d', 'COV_low_free_turnover_10_TsNorm30d', 'COV_close_vwap_10_TsNorm30d', 'COV_close_volume_10_TsNorm30d', 'COV_close_pct_chg_10_TsNorm30d', 'COV_close_turnover_10_TsNorm30d', 'COV_close_free_turnover_10_TsNorm30d', 'COV_vwap_volume_10_TsNorm30d', 'COV_vwap_pct_chg_

100%|██████████| 1286/1286 [01:10<00:00, 18.27it/s]


COV_open_high_10_TsNorm30d  \
timestamp           ticker                               
2016-03-22 09:30:00 601058                    4.683169   
                    600883                    4.496012   
                    600570                    4.417500   
                    300073                    4.083045   
                    300122                    3.962235   
...                                                ...   
2021-05-31 09:30:00 300190                   -1.919993   
                    300650                   -1.932960   
                    000795                   -1.934809   
                    600160                   -1.983296   
                    003022                   -1.996742   

                            COV_open_low_10_TsNorm30d  \
timestamp           ticker                              
2016-03-22 09:30:00 601058                   4.486948   
                    600883                   4.387932   
                    600570                   4.507623   
                    300073                   4.170303   
                    300122                   3.460923   
...                                               ...   
2021-05-31 09:30:00 300190                  -1.419622   
                    300650                  -1.867158   
                    000795                  -2.279957   
                    600160                  -1.617347   
                    003022                  -1.908923   

                            COV_open_close_10_TsNorm30d  \
timestamp           ticker                                
2016-03-22 09:30:00 601058                     4.602206   
                    600883                     4.326616   
                    600570                     4.475134   
                    300073                     4.020029   
                    300122                     4.056136   
...                                                 ...   
2021-05-31 09:30:00 300190                    -1.600213   
                    300650                    -1.857578   
                    000795                    -2.364901   
                    600160                    -1.711656   
                    003022                    -1.830822   

                            COV_open_vwap_10_TsNorm30d  \
timestamp           ticker                               
2016-03-22 09:30:00 601058                    4.579961   
                    600883                    4.399336   
                    600570                    4.472461   
                    300073                    4.087482   
                    300122                    3.771486   
...                                                ...   
2021-05-31 09:30:00 300190                   -1.640009   
                    300650                   -1.902845   
                    000795                   -2.011395   
                    600160                   -1.828793   
                    003022                   -2.030047   

                            COV_open_volume_10_TsNorm30d  \
timestamp           ticker                                 
2016-03-22 09:30:00 601058                      5.083368   
                    600883                      4.592056   
                    600570                      2.021638   
                    300073                      3.992399   
                    300122                      4.844355   
...                                                  ...   
2021-05-31 09:30:00 300190                     -0.671004   
                    300650                     -0.637708   
                    000795                     -1.347145   
                    600160                     -1.354330   
                    003022                      0.343931   

                            COV_open_pct_chg_10_TsNorm30d  \
timestamp           ticker                                  
2016-03-22 09:30:00 601058                       3.795460   
                    600883                  

# Shift

In [10]:
generate_shift_data(alpha_name=alpha_name,shift=shift,sequence=sequence,target=target)

100%|██████████| 1705/1705 [00:27<00:00, 61.40it/s] 


[25, 20, 15, 10, 5, 0]


100%|██████████| 1261/1261 [06:16<00:00,  3.35it/s]


COV_open_high_10_TsNorm30d_Shift_25  \
timestamp           ticker                                        
2016-03-25 09:30:00 000001                            -1.064060   
                    000005                            -0.494749   
                    000006                            -0.069611   
                    000008                            -0.923591   
                    000009                            -0.482581   
...                                                         ...   
2021-05-31 09:30:00 688698                             0.221239   
                    688777                            -0.832943   
                    688788                             0.663729   
                    688819                             0.997056   
                    688981                            -1.046654   

                            COV_open_low_10_TsNorm30d_Shift_25  \
timestamp           ticker                                       
2016-03-25 09:30:00 000001                           -1.005649   
                    000005                           -0.464317   
                    000006                           -0.019566   
                    000008                           -0.784274   
                    000009                           -0.464976   
...                                                        ...   
2021-05-31 09:30:00 688698                            0.565130   
                    688777                           -0.892889   
                    688788                            0.513295   
                    688819                            0.956463   
                    688981                           -1.090155   

                            COV_open_close_10_TsNorm30d_Shift_25  \
timestamp           ticker                                         
2016-03-25 09:30:00 000001                             -1.068242   
                    000005                             -0.483210   
                    000006                             -0.026334   
                    000008                             -0.750826   
                    000009                             -0.496014   
...                                                          ...   
2021-05-31 09:30:00 688698                              0.060212   
                    688777                             -0.845271   
                    688788                              0.477958   
                    688819                              1.082692   
                    688981                             -0.992281   

                            COV_open_vwap_10_TsNorm30d_Shift_25  \
timestamp           ticker                                        
2016-03-25 09:30:00 000001                            -1.037194   
                    000005                            -0.556732   
                    000006                            -0.050845   
                    000008                            -0.873589   
                    000009                            -0.485810   
...                                                         ...   
2021-05-31 09:30:00 688698                             0.257353   
                    688777                            -0.831188   
                    688788                             0.627277   
                    688819                             0.984160   
                    688981                            -1.053226   

                            COV_open_volume_10_TsNorm30d_Shift_25  \
timestamp           ticker                                          
2016-03-25 09:30:00 000001                              -0.093782   
                    000005                               2.117554   
                    000006                               2.650522   
                    000008                               0.104365   
                    000009                               1.196756   
...                                                      

# Task Generation

In [7]:
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()
original = task_info["Task"]
display(original)
# Timelist
time_list = ["2019-01-01","2019-06-01","2020-01-01","2020-06-01","2021-01-01","2021-06-01"]
time_list2 = []
for i in range(len(time_list)-1):
    time_list2.append([time_list[i],time_list[i+1]])
time_list2 = pd.DataFrame(time_list2,columns=["start_date","end_date"])
display(time_list2)

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
task_id,,,,,,,,,
0,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-01-01,2019-06-01,finished,None
1,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-06-01,2020-01-01,finished,None
2,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-01-01,2020-06-01,finished,None
3,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-06-01,2021-01-01,finished,None
4,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2021-01-01,2021-06-01,finished,None
...,...,...,...,...,...,...,...,...,...
85,AlphaNet_Moving5_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-01-01,2019-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
86,AlphaNet_Moving5_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-06-01,2020-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
87,AlphaNet_Moving5_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-01-01,2020-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."


,start_date,end_date
0,2019-01-01,2019-06-01
1,2019-06-01,2020-01-01
2,2020-01-01,2020-06-01
3,2020-06-01,2021-01-01
4,2021-01-01,2021-06-01


In [8]:
task = pd.DataFrame([
                    alpha_name + "_Shift_%i_Sequence_%i" % (shift,sequence),
                    sequence,LR,epoch_num,len(alpha_list),
                        ],index=["Alpha_Name","sequence","LR","epoch_num","feature_num"]).T
task['value']=1
time_list2['value']=1
task = pd.merge(task,time_list2,how='left',on='value')
del task['value']
task["status"] = "waiting"
task["description"] = [{
    "target" : target,
    "alpha_name" : alpha_name,
    "alpha_list" : alpha_list,
    "shift" : shift,
    "sequence" : sequence
} for i in range(len(task))]
task

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
0,AlphaNet_Moving5_Shift_3_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-01-01,2019-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
1,AlphaNet_Moving5_Shift_3_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-06-01,2020-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
2,AlphaNet_Moving5_Shift_3_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-01-01,2020-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
3,AlphaNet_Moving5_Shift_3_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-06-01,2021-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
4,AlphaNet_Moving5_Shift_3_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2021-01-01,2021-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."


In [9]:
task = pd.concat([original,task],axis=0)
task.reset_index(drop=True,inplace=True)
task.index.names = ["task_id"]
task_info["Task"] = task
task[-20:]

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
task_id,,,,,,,,,
75,Original_Input_Ts_Stand_Residual_Concated_Shif...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2019-01-01,2019-06-01,finished,"{'target': '10d_ret_standscalar', 'alpha_name'..."
76,Original_Input_Ts_Stand_Residual_Concated_Shif...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2019-06-01,2020-01-01,finished,"{'target': '10d_ret_standscalar', 'alpha_name'..."
77,Original_Input_Ts_Stand_Residual_Concated_Shif...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2020-01-01,2020-06-01,finished,"{'target': '10d_ret_standscalar', 'alpha_name'..."
78,Original_Input_Ts_Stand_Residual_Concated_Shif...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2020-06-01,2021-01-01,finished,"{'target': '10d_ret_standscalar', 'alpha_name'..."
79,Original_Input_Ts_Stand_Residual_Concated_Shif...,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",18,2021-01-01,2021-06-01,finished,"{'target': '10d_ret_standscalar', 'alpha_name'..."
80,AlphaNetV1_Ts_Standard_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-01-01,2019-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
81,AlphaNetV1_Ts_Standard_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-06-01,2020-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
82,AlphaNetV1_Ts_Standard_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-01-01,2020-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
83,AlphaNetV1_Ts_Standard_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-06-01,2021-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."


In [10]:
np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",task_info)
np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/Task.npy",task_info)